In [73]:
from ACDC.random_walk_classifier import * 
from ACDC.cell_type_annotation import * 

In [74]:
import pandas as pd
import numpy as np
from collections import Counter
import pickle

channels = ['Ly.6G.C','TNFa','CD45','B.Catenin','CD11b','p.Rb','CD64','p.Akt',
            'CyclinB1','TGFB','p.p38','p.Stat3','F4.80','ITGA7','Ly.6C','PDGFRA',
            'IFNg','IL.4','IL.6','CD31','Ly.6A.E..SCA1.','CD34','CD86','CD106..VCAM1.','p.H3']

path = 'data/muscle/muscle_40k/'

df = pd.read_csv(path + 'muscle_40k_5.csv.gz', sep=',', header = 0, compression = 'gzip')
#df = df[df.cell_type != 'NotGated']


table = pd.read_csv(path + 'muscle_table.csv', sep=',', header=0, index_col=0)
table = table.fillna(0)

cts, channels = get_label(table)

#X0= np.arcsinh((df[channels].values - 1.0)/5.0)
X0= df[channels].values

In [75]:
idx2ct = [key for idx, key in enumerate(table.index)]
#idx2ct.append('unknown')

ct2idx = {key:idx for idx, key in enumerate(table.index)}
#ct2idx['unknown'] = len(table.index)
        
ct_score = np.abs(table.as_matrix()).sum(axis = 1)

## compute manual gated label
y0 = np.zeros(df.cell_type.shape)

for i, ct in enumerate(df.cell_type):
    if ct in ct2idx:
        y0[i] = ct2idx[ct]
    #else:
        #y0[i] = ct2idx['unknown']

In [76]:
from sklearn.metrics import accuracy_score, confusion_matrix
import phenograph
from sklearn.cross_validation import StratifiedKFold

n_neighbor = 10
thres = 0.5


In [77]:
import time
import scipy.io as sio 

skf = StratifiedKFold(y0, n_folds=5, shuffle=True, random_state=0)
result = []
score_final = []


process_time = []
c = 0
for tr, te in skf:
    print('%02d th batch' % c)
    if c == 1:
        break
    c += 1
    
    X = X0.copy()
    y_true = y0.copy()

    X = X[tr, :]
    y_true = y_true[tr]

    mk_model =  compute_marker_model(pd.DataFrame(X, columns = channels), table, 0.0)

    ## compute posterior probs
    tic = time.clock()
    score = get_score_mat(X, [], table, [], mk_model)
    score = np.concatenate([score, 1.0 - score.max(axis = 1)[:, np.newaxis]], axis = 1)    

    ## get indices     
    ct_index = get_unique_index(X, score, table, thres)
    
    ## baseline - classify events    
    y_pred_index = np.argmax(score, axis = 1)
    
    toc = time.clock()
    time0 = toc - tic
    
    
    
    ## running ACDC
    tic = time.clock()
    res_c = get_landmarks(X, score, ct_index, idx2ct, phenograph, thres)

    landmark_mat, landmark_label = output_feature_matrix(res_c, [idx2ct[i] for i in range(len(idx2ct))]) 

    landmark_label = np.array(landmark_label)

    lp, y_pred = rm_classify(X, landmark_mat, landmark_label, n_neighbor)

    process_time.append(toc-tic)
    
    res = phenograph.cluster(X, k=30, directed=False, prune=False, min_cluster_size=10, jaccard=True,
                        primary_metric='euclidean', n_jobs=-1, q_tol=1e-3)
    
    toc = time.clock()
    time1 = toc - tic
    
    
    ## running phenograph classification
    tic = time.clock()
    y_pred_oracle = np.zeros_like(y_true)
    for i in range(max(res[0])+1):
        ic, nc = Counter(y_true[res[0] == i]).most_common(1)[0]
        y_pred_oracle[res[0] == i] = ic
        
    score_final.append([accuracy_score(y_true, [ct2idx[c] for c in y_pred]), 
                    accuracy_score(y_true, y_pred_index), 
                    accuracy_score(y_true, y_pred_oracle)])
    
    toc = time.clock()
    time2 = toc - tic   
    
    
    result.append((y_true, y_pred, y_pred_index, y_pred_oracle))
    process_time.append((time0, time1, time2))
    
    #pickle.dump(result, open('processed_file/BMMC/event_classidication_BMMC.p', 'wb'))
    sio.savemat('processed_file/muscle/muscle40k/event_classidication_muscle_5.mat',{'y_true':y_true,'y_pred_index':y_pred_index,'y_pred_oracle':y_pred_oracle,'X':X})

00 th batch
Finding 30 nearest neighbors using minkowski metric and 'auto' algorithm
Neighbors computed in 2.380136013031006 seconds
Jaccard graph constructed in 2.756157636642456 seconds
Wrote graph to binary file in 1.0440597534179688 seconds
Running Louvain modularity optimization
After 1 runs, maximum modularity is Q = 0.748203
After 5 runs, maximum modularity is Q = 0.751569
After 12 runs, maximum modularity is Q = 0.752663
After 15 runs, maximum modularity is Q = 0.755161
Louvain completed 35 runs in 28.437626361846924 seconds
PhenoGraph complete in 34.672982931137085 seconds
Finding 30 nearest neighbors using minkowski metric and 'auto' algorithm
Neighbors computed in 0.5730328559875488 seconds
Jaccard graph constructed in 2.050117254257202 seconds
Wrote graph to binary file in 0.3760216236114502 seconds
Running Louvain modularity optimization
After 1 runs, maximum modularity is Q = 0.676725
After 3 runs, maximum modularity is Q = 0.679676
Louvain completed 23 runs in 8.67849636

In [78]:
np.mean(score_final, axis = 0) # score of ACDC, score-based classification, phenograph classification

array([ 0.89458057,  0.96230779,  0.86632704])

In [79]:
score_final

[[0.89458057257157142, 0.96230778847355924, 0.86632704088011003]]

In [80]:
process_time

[-1.7105967344832607e-06,
 (0.1269063174618168, 190.14501868541765, 0.03021683523638785)]